In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import missingno as msno
import random

df = pd.read_csv('Train_HR_Employee_Attrition.csv')

In [2]:
## DATA QUALITY ##

df.Gender = df.groupby(['JobRole','MaritalStatus'])['Gender'].apply(lambda x: x.fillna(x.mode().iloc[0]))

df.YearsAtCompany = df.groupby(['JobRole','Gender','MaritalStatus'])['YearsAtCompany'].apply(lambda x: x.fillna(x.mean()))

bt_dic = df.BusinessTravel.value_counts(normalize=True).to_dict()
def give_random():
    return random.choices(list(bt_dic),list(bt_dic.values()))[0]
df.loc[df.BusinessTravel.isnull(),'BusinessTravel'] = df.loc[df.BusinessTravel.isnull(),'BusinessTravel'].apply(lambda x: give_random())

df.loc[:,'PerformanceRating'] = df.groupby(['Education','JobLevel'])['PerformanceRating'].apply(lambda x: x.fillna(x.mode().iloc[0]))

df.loc[:,'Age']=df.groupby(['Gender','JobSatisfaction','EnvironmentSatisfaction','StockOptionLevel'])['Age'].apply(lambda x: x.fillna(x.mean()))

df.loc[:,'Age_cat'] = pd.cut(df.Age,
                                bins=[17,25,35,45,55,60],
                                labels=['Very low','low','Medium','High','Very High'])
df.loc[:,'MonthlyIncome'] = df.groupby(['Gender', 'Age_cat'])['MonthlyIncome'].apply(lambda x: x.fillna(x.mean()))

df.drop(columns = 'TrainingTimesLastYear', inplace = True)

#Elimino due colonne irrilevanti #addByAgne
df.drop(columns=['StandardHours','Over18', 'Age_cat'], inplace = True)

In [3]:
## TRANSFORMATON ##

#Trasformazione in binari di tre attributi categorici 
df['Attrition'] = (df['Attrition'] == "Yes").astype(int)
df['OverTime'] = (df['OverTime'] == "Yes").astype(int)
df['Gender'] = (df['Gender'] == "Male").astype(int)

#Discretizzazione della variabile DistanceFromHome
for i, row in df.iterrows():
    if row['DistanceFromHome'] <= 8: #Proximate
        df.loc[i, 'DistanceFromHome'] = 1 
    if row['DistanceFromHome'] > 8 and row['DistanceFromHome'] <= 15: #Near
        df.loc[i, 'DistanceFromHome'] = 2
    if row['DistanceFromHome'] > 15 and row['DistanceFromHome'] <= 22: #Intermediate
        df.loc[i, 'DistanceFromHome'] = 3
    if row['DistanceFromHome'] > 22 and row['DistanceFromHome'] <= 29: #Far
        df.loc[i, 'DistanceFromHome'] = 4

#Normalizzazione
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler().fit(df[['Age', 'DailyRate','HourlyRate','MonthlyRate','MonthlyIncome','PercentSalaryHike','YearsAtCompany']])
df_std = std_scale.transform(df[['Age', 'DailyRate','HourlyRate','MonthlyRate','MonthlyIncome','PercentSalaryHike','YearsAtCompany']])

df_new = pd.DataFrame(df_std)

#df_new.columns
df['Age'] = df_new[0].values
df['DailyRate'] = df_new[1].values
df['HourlyRate'] = df_new[2].values
df['MonthlyRate'] = df_new[3].values
df['MonthlyIncome'] = df_new[4].values
df['PercentSalaryHike'] = df_new[5].values
df['YearsAtCompany'] = df_new[6].values

In [4]:
## CORRELATION ##

for i, row in df.iterrows():
    if row['JobRole'] == 'Manager':
        if row['Department'] == 'Sales':
            df.loc[i, 'JobRole'] = 'Manager Sales'
        if row['Department'] == 'Human Resources':
            df.loc[i, 'JobRole'] =  'Manager HR'
        if row['Department'] == 'Research & Development':
            df.loc[i, 'JobRole'] =  'Manager R&D'
            
df.drop(columns = 'Department', inplace=True)

In [6]:
df.to_csv('Datasurce1.0.csv') #sistema di numerazione in previsione delle modifiche future, per essere tutti al pari